# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [1]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.4 MB/s eta 0:00:00


In [2]:
import optuna

optuna.__version__

'3.1.1'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [3]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection

def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.
    
    clf = sklearn.ensemble.RandomForestClassifier(    
        n_estimators=5, max_depth=3)  # Define the model.
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()  # Train and evaluate the model.

print('Accuracy: {}'.format(objective()))

Accuracy: 0.9466666666666667


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [4]:
import optuna

def objective(trial):
    iris = sklearn.datasets.load_iris()
    
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))
    
    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth)
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2023-05-15 15:02:19,472] A new study created in memory with name: no-name-aa0c6745-f783-4588-b7f1-68564e2efec8
[I 2023-05-15 15:02:19,540] Trial 0 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 12, 'max_depth': 2.36862128058673}. Best is trial 0 with value: 0.9666666666666667.
[I 2023-05-15 15:02:19,599] Trial 1 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 10, 'max_depth': 27.55807606323762}. Best is trial 0 with value: 0.9666666666666667.
[I 2023-05-15 15:02:19,687] Trial 2 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 18, 'max_depth': 3.651223368412711}. Best is trial 0 with value: 0.9666666666666667.
[I 2023-05-15 15:02:19,756] Trial 3 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 11, 'max_depth': 3.4533331396998808}. Best is trial 0 with value: 0.9666666666666667.
[I 2023-05-15 15:02:19,837] Trial 4 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 6

Accuracy: 0.9666666666666667
Best hyperparameters: {'n_estimators': 12, 'max_depth': 2.36862128058673}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [5]:
import sklearn.svm

def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 2, 20)
        max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2023-05-15 15:02:28,706] A new study created in memory with name: no-name-b75b8374-8496-409c-8cb2-640f3d613d8f
[I 2023-05-15 15:02:28,735] Trial 0 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 1.566942283208742e-05}. Best is trial 0 with value: 0.32.
[I 2023-05-15 15:02:28,768] Trial 1 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 6.014786734533924e-09}. Best is trial 0 with value: 0.32.
[I 2023-05-15 15:02:28,927] Trial 2 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 18, 'max_depth': 15.048040635627752}. Best is trial 2 with value: 0.96.
[I 2023-05-15 15:02:28,946] Trial 3 finished with value: 0.96 and parameters: {'classifier': 'SVC', 'svc_c': 914025286.8936354}. Best is trial 2 with value: 0.96.
[I 2023-05-15 15:02:28,970] Trial 4 finished with value: 0.96 and parameters: {'classifier': 'SVC', 'svc_c': 39139.398272292325}. Best is trial 2 with value: 0.96.
[I 2023-05-15 15:02:29,022] 

Accuracy: 0.9866666666666667
Best hyperparameters: {'classifier': 'SVC', 'svc_c': 4.618568509471278}


### Plotting the study

Plotting the optimization history of the study.

In [6]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [7]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [8]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])